# CreditOne Classification 
Class 5 Task 3

#### Import modules

In [2]:
#imports
#numpy,pandas,scipy,math,matplotlib
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt

#estimators
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from sklearn import tree
from sklearn.neural_network import MLPClassifier

#model metrics
from sklearn.model_selection import cross_val_score
import sklearn.metrics as skm

#cross validation
from sklearn.model_selection import train_test_split

#### import data

In [79]:
#data
df = pd.read_csv('df_processed.csv', header=0)

In [80]:
del df['Unnamed: 0']

In [81]:
df.head()

,ID,LIMIT_BAL,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,EDUCATION_3,EDUCATION_4,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,AGE_80-70,SEX_1,SEX_2
0,4810,500000,0,0,0,0,0,0,415022,421996,...,1,0,0,0,0,1,0,0,1,0
1,15096,500000,-2,-2,-2,-2,-2,-2,1523,0,...,0,0,0,1,0,0,0,0,1,0
2,81,470000,0,0,0,0,0,0,165254,157784,...,1,0,0,1,0,0,0,0,0,1
3,14889,450000,1,-2,-2,-2,-2,-2,0,0,...,0,0,0,1,0,0,0,0,0,1
4,11072,450000,-2,-2,-2,-2,-2,-2,324,302,...,0,0,0,1,0,0,0,0,1,0


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 37 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6                     30000 non-null int64
PAY_AMT1                      30000 non-null int64
PAY_AMT2                      30000 non-null int64
PAY_AMT3                      30000 non-null int64
PAY_AMT4                

In [85]:
#move default column to end

#make an array of column names
cols = df.columns.tolist()

#move default to the end
cols.insert(36, cols.pop(cols.index('default payment next month')))
cols

['ID',
 'LIMIT_BAL',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'MARRIAGE_0',
 'MARRIAGE_1',
 'MARRIAGE_2',
 'MARRIAGE_3',
 'EDUCATION_1',
 'EDUCATION_2',
 'EDUCATION_3',
 'EDUCATION_4',
 'AGE_20-30',
 'AGE_30-40',
 'AGE_40-50',
 'AGE_50-60',
 'AGE_60-70',
 'AGE_80-70',
 'SEX_1',
 'SEX_2',
 'default payment next month']

In [93]:
#change to categorical

#df.SEX = df.SEX.astype('category')
df.EDUCATION_4 = df.EDUCATION_4.astype('category')
#df.MARRIAGE = df.MARRIAGE.astype('category')
#df.AGE = df.AGE_20-30.astype('category')

#### Select Features


### Remove low variance features

In [27]:
from sklearn.feature_selection import VarianceThreshold

In [31]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(df)

array([[  4810, 500000,      0, ...,      0,      1,      0],
       [ 15096, 500000,     -2, ...,      0,      1,      0],
       [    81, 470000,      0, ...,      0,      0,      1],
       ..., 
       [ 16582,  20000,      5, ...,      0,      1,      0],
       [ 21560,  20000,      0, ...,      0,      0,      1],
       [  6390,  10000,      0, ...,      0,      1,      0]])

In [29]:
df.shape

(30000, 37)

In [95]:
# features (didn't use variance....)

features = df.iloc[:,1:36]
print('Summary of feature sample')
features.head()

Summary of feature sample


,LIMIT_BAL,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,EDUCATION_2,EDUCATION_3,EDUCATION_4,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,AGE_80-70,SEX_1
0,500000,0,0,0,0,0,0,415022,421996,426103,...,0,1,0,0,0,0,1,0,0,1
1,500000,-2,-2,-2,-2,-2,-2,1523,0,0,...,0,0,0,0,1,0,0,0,0,1
2,470000,0,0,0,0,0,0,165254,157784,162702,...,0,1,0,0,1,0,0,0,0,0
3,450000,1,-2,-2,-2,-2,-2,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,450000,-2,-2,-2,-2,-2,-2,324,302,5018,...,0,0,0,0,1,0,0,0,0,1


In [96]:
features.shape

(30000, 35)

#### Select Dependent Variable

In [97]:
#dependent variable
depVar = df['default payment next month']

In [98]:
depVar.describe()

count    30000.000000
mean         0.221200
std          0.415062
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: default payment next month, dtype: float64

### Establish Training Data


In [99]:
#run split
X_train, X_test, y_train, y_test = train_test_split(features, depVar, test_size = 0.30, random_state =123)
print("X_train",X_train.shape,'\n',"X_test", X_test.shape,'\n',"y_train", y_train.shape,'\n',"y_test", y_test.shape)

X_train (21000, 35) 
 X_test (9000, 35) 
 y_train (21000,) 
 y_test (9000,)


In [100]:
X_test.describe()

,LIMIT_BAL,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,AGE_80-70,SEX_1
count,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,...,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000
mean,166248.186667,-0.014222,-0.112889,-0.151333,-0.206778,-0.252000,-0.280111,50839.135444,48589.317000,46512.621111,...,0.459444,0.529000,0.010222,0.367778,0.355667,0.200111,0.067333,0.008667,0.000444,0.398778
std,129030.764200,1.128492,1.209167,1.205051,1.182539,1.143084,1.165056,73012.738740,69928.882208,67853.283672,...,0.498380,0.499186,0.100593,0.482227,0.478741,0.400106,0.250612,0.092696,0.021078,0.489674
min,10000.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-11545.000000,-69777.000000,-157264.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3601.750000,3126.500000,2919.750000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,140000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21941.000000,20921.000000,19996.500000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,232500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67273.750000,63824.750000,59628.250000,...,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,780000.000000,8.000000,7.000000,8.000000,8.000000,8.000000,8.000000,653062.000000,671563.000000,693131.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [109]:
X_train.describe()

,LIMIT_BAL,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,AGE_80-70,SEX_1
count,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,2.100000e+04,...,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000
mean,168014.095238,-0.017762,-0.142714,-0.172571,-0.226619,-0.272286,-0.295810,51387.986095,49431.828667,4.722767e+04,...,0.453524,0.533476,0.011000,0.366810,0.357714,0.200190,0.066238,0.008524,0.000524,0.395190
std,130053.159397,1.121810,1.191931,1.193315,1.163326,1.128892,1.143465,73902.423155,71700.839216,6.998128e+04,...,0.497847,0.498890,0.104305,0.481945,0.479339,0.400152,0.248704,0.091932,0.022881,0.488903
min,10000.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-33350.000000,-6.150600e+04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3527.750000,2913.750000,2.585250e+03,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,140000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22476.000000,21335.500000,2.014450e+04,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,240000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67070.500000,64122.750000,6.037675e+04,...,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1000000.000000,8.000000,8.000000,8.000000,7.000000,7.000000,8.000000,964511.000000,983931.000000,1.664089e+06,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [102]:
y_test.describe()

count    9000.000000
mean        0.222778
std         0.416133
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: default payment next month, dtype: float64

In [103]:
y_train.describe()

count    21000.000000
mean         0.220524
std          0.414610
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: default payment next month, dtype: float64

# SVM
http://scikit-learn.org/stable/modules/svm.html#classification

In [104]:
SVMclf = svm.SVC()

In [105]:
SVMclf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [106]:
SVMclf_pred = SVMclf.predict(X_test)

In [107]:
print(" ***Support Vector Machine***","\n","accuracy",skm.accuracy_score(y_test, SVMclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, SVMclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, SVMclf_pred),'\n','Precission Score',skm.precision_score(y_test,SVMclf_pred))

 ***Support Vector Machine*** 
 accuracy 0.781555555556 
 average_precision_score 0.237702489807 
 Kappa 0.0362414396572 
 Precission Score 0.809523809524


In [52]:
SVMclf.score(X_test, y_test) 

0.78155555555555556

# Stochastic Gradient Descent

http://scikit-learn.org/stable/modules/sgd.html#classification

In [53]:
SDGclf = SGDClassifier(loss="hinge", penalty="l2")

In [54]:
SDGclf.fit(X_train, y_train)

/Users/degraff/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [55]:
SDGclf_pred = SDGclf.predict(X_test)

In [56]:
print(" ***Stochastic Gradient Descent***","\n","accuracy",skm.accuracy_score(y_test, SDGclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, SDGclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, SDGclf_pred),'\n','Precission Score',skm.precision_score(y_test,SDGclf_pred))

 ***Stochastic Gradient Descent*** 
 accuracy 0.776555555556 
 average_precision_score 0.222729010806 
 Kappa -0.000778549200107 
 Precission Score 0.125


In [57]:
SDGclf.score(X_test, y_test) 

0.77655555555555555

# Nearest Centroid Classifier

http://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification

In [59]:
NCCclf = NearestCentroid()

In [60]:
NCCclf.fit(X_train, y_train)

NearestCentroid(metric='euclidean', shrink_threshold=None)

In [61]:
NCCclf_pred = NCCclf.predict(X_test)

In [62]:
print(" ***Nearest Centroid Classifier***","\n","accuracy",skm.accuracy_score(y_test, NCCclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, NCCclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, NCCclf_pred),'\n','Precision Score',skm.precision_score(y_test,NCCclf_pred))

 ***Nearest Centroid Classifier*** 
 accuracy 0.531333333333 
 average_precision_score 0.257792545966 
 Kappa 0.108053947205 
 Precision Score 0.274781192754


In [90]:
#same as accuracy_score(y_test,clf_pred)
NCCclf.score(X_test, y_test) 

0.53133333333333332

# K-Nearest Neighbor Classifier

http://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification

In [63]:
KNNclf = KNeighborsClassifier(n_neighbors=30)

In [64]:
KNNclf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=30, p=2,
           weights='uniform')

In [65]:
KNNclf.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': 1,
 'n_neighbors': 30,
 'p': 2,
 'weights': 'uniform'}

In [66]:
KNNclf_pred = KNNclf.predict(X_test)

In [67]:
print(" ***K-Nearest Neighbor Classifier***","\n","accuracy",skm.accuracy_score(y_test, KNNclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, KNNclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, KNNclf_pred),'\n','Precision Score',skm.precision_score(y_test,KNNclf_pred))

 ***K-Nearest Neighbor Classifier*** 
 accuracy 0.778444444444 
 average_precision_score 0.245493026693 
 Kappa 0.0809662298875 
 Precision Score 0.518518518519


In [87]:
KNNclf.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': 1,
 'n_neighbors': 30,
 'p': 2,
 'weights': 'uniform'}

In [91]:
KNNclf.score(X_test, y_test) 

0.77666666666666662

# Decision Tree Classifer

http://scikit-learn.org/stable/modules/tree.html#classification

In [68]:
DTclf = tree.DecisionTreeClassifier()

In [72]:
DTclf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [73]:
DTclf_pred = DTclf.predict(X_test)

In [74]:
print(" ***Decision Tree Classifier***" ,'\n',"accuracy",skm.accuracy_score(y_test, DTclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, DTclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, DTclf_pred),'\n','Precision Score',skm.precision_score(y_test,DTclf_pred))

 ***Decision Tree Classifier*** 
 accuracy 1.0 
 average_precision_score 1.0 
 Kappa 1.0 
 Precision Score 1.0


# Multi-layer Perceptron

http://scikit-learn.org/stable/modules/neural_networks_supervised.html#classification

In [75]:
MLPclf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100,), random_state=1)

In [76]:
MLPclf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [77]:
MLPclf_pred = MLPclf.predict(X_test)

In [78]:
print(" ***Multi-layer Perceptron***" ,'\n',"accuracy",skm.accuracy_score(y_test, MLPclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, MLPclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, MLPclf_pred),'\n','Precision Score',skm.precision_score(y_test,MLPclf_pred))

 ***Multi-layer Perceptron*** 
 accuracy 0.749 
 average_precision_score 0.257281443267 
 Kappa 0.137198617888 
 Precision Score 0.385171790235


## Making Predictions

# Tuning

In [95]:
scores = cross_val_score(SVMclf, X_train, y_train, cv=15)
scores

array([ 0.78087081,  0.77872948,  0.78301213,  0.77944325,  0.77785714,
        0.77714286,  0.78      ,  0.78      ,  0.77857143,  0.78428571,
        0.78142857,  0.78270193,  0.77698356,  0.78055754,  0.78198713])

In [96]:
scores = cross_val_score(SDGclf, X_train, y_train, cv=15)
scores

/Users/degraff/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/degraff/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/degra

array([ 0.77872948,  0.76802284,  0.64882227,  0.77944325,  0.49071429,
        0.77928571,  0.77142857,  0.77928571,  0.77571429,  0.75      ,
        0.77928571,  0.71265189,  0.77984274,  0.64903503,  0.77841315])

In [94]:
scores = cross_val_score(NCCclf, X_train, y_train, cv=15)
scores

array([ 0.5267666 ,  0.52462527,  0.53961456,  0.54246966,  0.54714286,
        0.56357143,  0.50857143,  0.52642857,  0.53071429,  0.53571429,
        0.54357143,  0.5196569 ,  0.5382416 ,  0.5375268 ,  0.53323803])

In [94]:
scores = cross_val_score(DTclf, X_train, y_train, cv=15)
scores

array([ 0.5267666 ,  0.52462527,  0.53961456,  0.54246966,  0.54714286,
        0.56357143,  0.50857143,  0.52642857,  0.53071429,  0.53571429,
        0.54357143,  0.5196569 ,  0.5382416 ,  0.5375268 ,  0.53323803])

In [94]:
scores = cross_val_score(NCCclf, X_train, y_train, cv=15)
scores

array([ 0.5267666 ,  0.52462527,  0.53961456,  0.54246966,  0.54714286,
        0.56357143,  0.50857143,  0.52642857,  0.53071429,  0.53571429,
        0.54357143,  0.5196569 ,  0.5382416 ,  0.5375268 ,  0.53323803])